In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# chen2020mvlidarnet

> Module that implements the models from [MVLidarNet: Real-Time Multi-Class Scene Understanding for Autonomous Driving Using Multiple Views](https://arxiv.org/pdf/2006.05518).

**(UNDER CONSTRUCTION...)**

In [ ]:
#| default_exp chen2020mvlidarnet

In [ ]:
#| export
import torch
from torch.nn import Module, Sequential, Conv2d, BatchNorm2d, ReLU, ConvTranspose2d, MaxPool2d
from torch.nn.init import kaiming_normal_, constant_, zeros_
from collections import OrderedDict
import re

In [ ]:
#| export
class ConvBNReLU(Sequential):
    "Sequential composition of convolution, batch normalization and ReLU."
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, has_ReLU=True):
        sequence = [
            ('conv', Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)),
            ('bn', BatchNorm2d(out_channels, momentum=0.01))
        ]
        self.nonlinearity = 'linear'
        if has_ReLU:
            self.nonlinearity = 'relu'
            sequence += [('relu', ReLU())]
        super().__init__(OrderedDict(sequence))
        self.init_params()

    def init_params(self):
        for n, p in self.named_parameters():
            if re.search('conv\.weight', n):
                kaiming_normal_(p, nonlinearity=self.nonlinearity)
            if re.search('bn\.bias', n):
                constant_(p, 0.1)

In [ ]:
bs, in_c, out_c, h, w = 1, 5, 64, 64, 2048
inp = torch.randn(bs, in_c, h, w)

b = ConvBNReLU(in_c, out_c, 3, 1, 1)
outp = b(inp)
assert outp.shape == (bs, out_c, h, w)
print(outp.shape, f'== ({bs}, {out_c}, {h}, {w})')

torch.Size([1, 64, 64, 2048]) == (1, 64, 64, 2048)


In [ ]:
#| export
class InceptionV2(Module):
    "InceptionV2 Block from [Rethinking the Inception Architecture for Computer Vision](https://arxiv.org/pdf/1512.00567)."
    def __init__(self, in_channels, out_channels):
        super().__init__()
        out_channels = out_channels//4
        
        # 1x1
        self.b1 = ConvBNReLU(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        
        # pool -> 1x1
        self.b2 = Sequential(OrderedDict([
            ('pool', MaxPool2d(kernel_size=3 , stride=1 , padding=1)),
            ('cbr', ConvBNReLU(in_channels, out_channels, kernel_size=1, stride=1, padding=0))
        ]))
        
        # 1x1 -> 3x3
        self.b3 = Sequential(OrderedDict([
            ('cbr1', ConvBNReLU(in_channels, out_channels, kernel_size=1, stride=1, padding=0)),
            ('cbr2', ConvBNReLU(out_channels, out_channels, kernel_size=3, stride=1, padding=1))
        ]))
        
        # 1x1 -> 3x3 -> 3x3
        self.b4 = Sequential(OrderedDict([
            ('cbr1', ConvBNReLU(in_channels, out_channels, kernel_size=1, stride=1, padding=0)),
            ('cbr2', ConvBNReLU(out_channels, out_channels, kernel_size=3, stride=1, padding=1)),
            ('cbr3', ConvBNReLU(out_channels, out_channels, kernel_size=3, stride=1, padding=1))
        ]))
    
    def forward(self, x):
        branch1 = self.b1(x)
        branch2 = self.b2(x)
        branch3 = self.b3(x)
        branch4 = self.b4(x)
        return torch.cat((branch1, branch2, branch3, branch4), dim=1)

In [ ]:
b = InceptionV2(in_c, out_c)
outp = b(inp)
assert outp.shape == (bs, out_c, h, w)
print(outp.shape, f'== ({bs}, {out_c}, {h}, {w})')

torch.Size([1, 64, 64, 2048]) == (1, 64, 64, 2048)


In [ ]:
#| export
class InceptionBlock(Sequential):
    "Sequential composition of InceptionV2 modules."
    def __init__(self, in_channels, out_channels, n_modules, has_pool=False):
        modules_in_block = []
        for i in range(n_modules):
            modules_in_block += [(f'incept{i+1}', InceptionV2(in_channels, out_channels))]
            in_channels = out_channels
        if has_pool:
            modules_in_block += [('pool', MaxPool2d(3, 2, 1))] # kernel_size = 3, because of inception v2 paper table 1
        super().__init__(OrderedDict(modules_in_block))

In [ ]:
b = InceptionBlock(in_c, out_c, 2)
outp = b(inp)
assert outp.shape == (bs, out_c, h, w)
print(outp.shape, f'== ({bs}, {out_c}, {h}, {w})')

torch.Size([1, 64, 64, 2048]) == (1, 64, 64, 2048)


In [ ]:
#| export
class Encoder(Module):
    "MVLidarNet encoder architecture."
    def __init__(self, in_channels=5):
        super().__init__()

        self.trunk1 = ConvBNReLU(in_channels, 64, kernel_size=3, stride=1, padding=1)
        self.trunk2 = ConvBNReLU(64, 64, kernel_size=3, stride=1, padding=1)
        self.trunk3 = Sequential(
            ConvBNReLU(64, 128, kernel_size=3, stride=1, padding=1),
            MaxPool2d(3, 2, 1) # kernel_size = 3, because of inception v2 paper table 1
        )
        
        self.block1 = InceptionBlock(in_channels=128, out_channels=64, n_modules=2)
        self.block2 = InceptionBlock(in_channels=64, out_channels=64, n_modules=2, has_pool=True)
        self.block3 = InceptionBlock(in_channels=64, out_channels=128, n_modules=3, has_pool=True)
    
    def forward(self, x):
        enc_features = [x]
        
        x = self.trunk1(x)
        x = self.trunk2(x)
        x = self.trunk3(x)
        x = self.block1(x)
        enc_features.append(x)
        
        x = self.block2(x)
        enc_features.append(x)
        
        x = self.block3(x)
        enc_features.append(x)
        
        return enc_features

In [ ]:
enc = Encoder()
outp = enc(inp)
[o.shape for o in outp]

[torch.Size([1, 5, 64, 2048]),
 torch.Size([1, 64, 32, 1024]),
 torch.Size([1, 64, 16, 512]),
 torch.Size([1, 128, 8, 256])]

In [ ]:
#| export
class Decoder(Module):
    "MVLidarNet decoder architecture."
    def __init__(self):
        super().__init__()
        self.up1a = ConvTranspose2d(128, 256, kernel_size=3, stride=2, padding=1, bias=False)
        self.up1c = ConvBNReLU(256+64, 256, kernel_size=1, stride=1, padding=0)
        self.up1d = ConvBNReLU(256, 256, kernel_size=3, stride=1, padding=1)
        
        self.up2a = ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.up2c = ConvBNReLU(128+64, 128, kernel_size=1, stride=1, padding=0)
        self.up2d = ConvBNReLU(128, 128, kernel_size=3, stride=1, padding=1)

        self.up3a = ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.up3b = ConvBNReLU(64, 64, kernel_size=1, stride=1, padding=0)
        self.up3c = ConvBNReLU(64, 64, kernel_size=3, stride=1, padding=1)
        self.init_params()

    def init_params(self):
        for n, p in self.named_parameters():
            if re.search('up\da\.weight', n):
                kaiming_normal_(p, nonlinearity='linear')
    
    def forward(self, enc_features):
        x = enc_features[-1]
        
        x = self.up1a(x, output_size=enc_features[-2].size())
        up1b = torch.cat((x, enc_features[-2]), dim=1)
        x = self.up1c(up1b)
        x = self.up1d(x)
        
        x = self.up2a(x, output_size=enc_features[-3].size())
        up2b = torch.cat((x, enc_features[-3]), dim=1)
        x = self.up2c(up2b)
        x = self.up2d(x)
        
        x = self.up3a(x, output_size=enc_features[-4].size())
        x = self.up3b(x)
        x = self.up3c(x)
        
        return x

In [ ]:
dec = Decoder()
fts = dec(outp)
assert fts.shape == (bs, out_c, h, w)
print(fts.shape, f'== ({bs}, {out_c}, {h}, {w})')

torch.Size([1, 64, 64, 2048]) == (1, 64, 64, 2048)


In [ ]:
#| export
class MVLidarNet(Module):
    "MVLidarNet semantic segmentation architecture."
    def __init__(self, in_channels=5, n_classes=7):
        super().__init__()
        self.n_classes = n_classes
        self.input_norm = BatchNorm2d(in_channels, affine=False, momentum=None)
        self.backbone = Sequential(OrderedDict([
            ('enc', Encoder(in_channels)),
            ('dec', Decoder())
        ]))
        
        self.classhead1 = ConvBNReLU(64, 64, kernel_size=3, stride=1, padding=1)
        self.classhead2 = ConvBNReLU(64, n_classes, kernel_size=1, stride=1, padding=0, has_ReLU=False)
    
    def forward(self, x):
        x = self.input_norm(x)
        features = self.backbone(x)
        x = self.classhead1(features)
        prediction = self.classhead2(x)
        return prediction

In [ ]:
n_classes=7
model = MVLidarNet()
logits = model(inp)
assert logits.shape == (bs, n_classes, h, w)
print(logits.shape, f'== ({bs}, {n_classes}, {h}, {w})')

torch.Size([1, 7, 64, 2048]) == (1, 7, 64, 2048)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()